In [1]:
import os
import random
import timm
import torch
import albumentations as A
import pandas as pd
import numpy as np
import torch.nn as nn
from albumentations.pytorch import ToTensorV2
from torch.optim import AdamW
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import StratifiedKFold
from PIL import Image
import wandb
from scipy.stats import mode

# 시드 고정
SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.benchmark = True

# WANDB 초기화
wandb.login()
wandb.init(project='v6_x50_class', name='fold-voting_SAR_attention')

# 디바이스 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 데이터 경로 및 설정
data_path_train = 'data/augmented_v3_x50/'
data_path_test = 'data/test/'
img_size = 224
LR = 1e-4
EPOCHS = 30
BATCH_SIZE = 16
num_workers = 16
patience = 3
WEIGHT_DECAY = 1e-4
DROPOUT_PROB = 0.4

# torchvision.transforms를 사용하여 추가적인 변형 정의
additional_transforms = T.Compose([
    T.RandomErasing(p=0.3, scale=(0.02, 0.33), ratio=(0.3, 3.3), value='random'),
])

# 데이터셋 클래스에서 추가적인 변형을 적용할 수 있도록 수정
class ImageDataset(Dataset):
    def __init__(self, df, path, transform=None, additional_transforms=None):
        self.df = df.values
        self.path = path
        self.transform = transform
        self.additional_transforms = additional_transforms

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        name, target = self.df[idx]
        img = Image.open(os.path.join(self.path, name)).convert("RGB")
        
        # 기본 transform 적용
        if self.transform:
            img = self.transform(image=np.array(img))['image']  # numpy 배열로 변환 후 transform 적용
        
        # 추가 변형 적용
        if self.additional_transforms:
            img = self.additional_transforms(img)
        
        return img, target

# SAR 모델 정의
class CustomSAR(nn.Module):
    def __init__(self, num_classes=17, dropout_prob=0.5, embed_dim=1024):
        super(CustomSAR, self).__init__()
        self.model = timm.create_model('resnet50', pretrained=True)
        num_features = self.model.fc.in_features
        
        # Attention 레이어를 위한 FC 레이어
        self.fc1 = nn.Linear(num_features, embed_dim)
        
        self.attention = nn.MultiheadAttention(embed_dim=embed_dim, num_heads=8, dropout=dropout_prob)
        
        # Attention 후 최종 분류를 위한 FC 레이어
        self.fc2 = nn.Sequential(
            nn.Linear(embed_dim, 1024),
            nn.ReLU(),
            nn.Dropout(dropout_prob),
            nn.Linear(1024, num_classes)
        )

    def forward(self, x):
        features = self.model.forward_features(x)  # ResNet의 특징 추출

        # Global Average Pooling 적용 (feature map을 평탄화하여 벡터로 변환)
        features = torch.mean(features, dim=[2, 3])  # [Batch Size, Channels]

        features = self.fc1(features)  # 임베딩 차원으로 변환

        # features의 차원을 [Seq Length, Batch Size, Embedding Dim]으로 맞춤
        features = features.unsqueeze(0)  # [1, Batch Size, Embedding Dim] 형태로 변경

        attn_output, _ = self.attention(features, features, features)
        attn_output = attn_output.squeeze(0)  # [Batch Size, Embedding Dim] 형태로 변환

        return self.fc2(attn_output)  # 최종 분류 레이어로 전달


# 학습 함수 정의
def train_one_epoch(loader, model, optimizer, loss_fn, device, scaler, clip_value=0.5):
    model.train()
    train_loss = 0
    preds_list = []
    targets_list = []

    pbar = tqdm(loader, desc="Training")
    for image, targets in pbar:
        image = image.to(device)
        targets = targets.to(device)

        optimizer.zero_grad()
        with torch.cuda.amp.autocast():
            preds = model(image)
            loss = loss_fn(preds, targets)
        
        scaler.scale(loss).backward()
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip_value)
        scaler.step(optimizer)
        scaler.update()

        train_loss += loss.item()
        preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())
        targets_list.extend(targets.detach().cpu().numpy())

        pbar.set_description(f"Loss: {loss.item():.4f}")

    train_loss /= len(loader)
    train_acc = accuracy_score(targets_list, preds_list)
    train_f1 = f1_score(targets_list, preds_list, average='macro')

    ret = {
        "train_loss": train_loss,
        "train_acc": train_acc,
        "train_f1": train_f1,
    }

    return ret

# 검증 함수 정의
def validate(loader, model, loss_fn, device):
    model.eval()
    val_loss = 0
    preds_list = []
    targets_list = []

    with torch.no_grad():
        for image, targets in tqdm(loader, desc="Validating"):
            image = image.to(device)
            targets = targets.to(device)

            preds = model(image)
            loss = loss_fn(preds, targets)

            val_loss += loss.item()
            preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())
            targets_list.extend(targets.detach().cpu().numpy())

    val_loss /= len(loader)
    val_acc = accuracy_score(targets_list, preds_list)
    val_f1 = f1_score(targets_list, preds_list, average='macro')

    ret = {
        "val_loss": val_loss,
        "val_acc": val_acc,
        "val_f1": val_f1,
    }

    return ret

# augmentation을 위한 transform 정의
trn_transform = A.Compose([
    A.Resize(height=img_size, width=img_size),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=15, p=0.5),
    A.CoarseDropout(p=0.5),
    A.GaussianBlur(p=0.3),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(),
])

# 데이터 로드
train_df_test = pd.read_csv('data/augment_v3_x50.csv')
test_df_test = pd.read_csv('data/sample_submission.csv')

# 클래스별 가중치 계산
class_counts = train_df_test['target'].value_counts().sort_index()
total_samples = len(train_df_test)
class_weights = [total_samples / class_counts[i] for i in range(len(class_counts))]
class_weights = np.array(class_weights)
class_weights = class_weights / class_weights.sum() * len(class_counts)
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)

# Stratified K-Fold 설정
n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=SEED)
folds = list(skf.split(train_df_test['ID'], train_df_test['target']))

# 학습 및 검증 루프
best_val_loss = float('inf')
fold_val_metrics = []
best_model_paths = []

for fold, (train_idx, val_idx) in enumerate(folds):
    print(f"Fold {fold + 1}")

    trn_dataset = ImageDataset(
        train_df_test.iloc[train_idx],
        data_path_train,
        transform=trn_transform,
        additional_transforms=additional_transforms  # 추가 변형 적용
    )

    val_dataset = ImageDataset(
        train_df_test.iloc[val_idx],
        data_path_train,
        transform=trn_transform  # 수정된 부분
    )

    trn_loader = DataLoader(
        trn_dataset,
        batch_size=BATCH_SIZE,
        shuffle=True,
        num_workers=num_workers,
        pin_memory=True,
        drop_last=False
    )

    val_loader = DataLoader(
        val_dataset,
        batch_size=BATCH_SIZE,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=True
    )

    # 모델 정의 (SAR로 변경)
    model = CustomSAR(num_classes=17, dropout_prob=DROPOUT_PROB).to(device)

    loss_fn = nn.CrossEntropyLoss(weight=class_weights)
    optimizer = AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.2, patience=2, verbose=True)
    scaler = torch.cuda.amp.GradScaler()

    fold_best_val_loss = float('inf')
    fold_best_model_path = f'best_model_fold_{fold + 1}_class_x50_SAR_attention.pth'
    patience_counter = 0

    for epoch in range(EPOCHS):
        train_metrics = train_one_epoch(trn_loader, model, optimizer, loss_fn, device, scaler)
        val_metrics = validate(val_loader, model, loss_fn, device)

        print(f"Epoch {epoch + 1}/{EPOCHS}, Train Loss: {train_metrics['train_loss']:.4f}, Train Acc: {train_metrics['train_acc']:.4f}, Train F1: {train_metrics['train_f1']:.4f}")
        print(f"Epoch {epoch + 1}/{EPOCHS}, Val Loss: {val_metrics['val_loss']:.4f}, Val Acc: {val_metrics['val_acc']:.4f}, Val F1: {val_metrics['val_f1']:.4f}")

        # WANDB 로그 기록
        wandb.log({
            'epoch': epoch + 1,
            'train_loss': train_metrics['train_loss'],
            'train_acc': train_metrics['train_acc'],
            'train_f1': train_metrics['train_f1'],
            'val_loss': val_metrics['val_loss'],
            'val_acc': val_metrics['val_acc'],
            'val_f1': val_metrics['val_f1']
        })

        scheduler.step(val_metrics['val_loss'])  # 수정된 부분

        if val_metrics['val_loss'] < fold_best_val_loss:
            fold_best_val_loss = val_metrics['val_loss']
            torch.save(model.state_dict(), fold_best_model_path)
            patience_counter = 0
        else:
            patience_counter += 1

        if patience_counter >= patience:
            print(f"Early stopping triggered at epoch {epoch + 1}")
            break

    best_model_paths.append(fold_best_model_path)
    fold_val_metrics.append((fold_best_val_loss, val_metrics['val_f1']))
    print(f"Fold {fold + 1} Macro F1 Score: {val_metrics['val_f1']:.4f}")

wandb.finish()



Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: kerynh (kerynhan). Use `wandb login --relogin` to force relogin


Fold 1


Validating: 100%|██████████| 982/982 [00:43<00:00, 22.47it/s]


Epoch 1/30, Train Loss: 0.8319, Train Acc: 0.7203, Train F1: 0.6999
Epoch 1/30, Val Loss: 0.2416, Val Acc: 0.9010, Val F1: 0.8862


Validating: 100%|██████████| 982/982 [00:57<00:00, 17.18it/s]


...

Validating: 100%|██████████| 982/982 [00:27<00:00, 35.98it/s]


Epoch 20/30, Train Loss: 0.0720, Train Acc: 0.9888, Train F1: 0.9881
Epoch 20/30, Val Loss: 0.0182, Val Acc: 0.9963, Val F1: 0.9962


Validating: 100%|██████████| 982/982 [00:27<00:00, 35.96it/s]


Epoch 21/30, Train Loss: 0.0630, Train Acc: 0.9902, Train F1: 0.9897
Epoch 21/30, Val Loss: 0.0286, Val Acc: 0.9961, Val F1: 0.9960
Epoch 00021: reducing learning rate of group 0 to 2.0000e-05.
Early stopping triggered at epoch 21
Fold 5 Macro F1 Score: 0.9960


epoch,▁▁▂▃▃▁▂▂▃▃▄▅▅▆▇▇▁▂▂▃▃▁▂▃▃▄▅▅▁▂▂▃▃▄▅▆▆▇▇█
train_acc,▁▆▇██▁▇▇████████▁▇▇██▆▇█████▁▇▇█████████
train_f1,▁▆▇██▁▇▇████████▁▇▇██▆▇█████▁▇▇█████████
train_loss,█▃▂▂▂█▂▂▂▂▁▁▁▁▁▁█▂▂▂▁▃▂▂▂▁▁▁█▂▂▂▂▁▁▁▁▁▁▁
val_acc,▁▅▇▇▇▁▆▇▇███████▁▆███▄▇▇▇███▂▆▇█████████
val_f1,▁▅▇▇▇▁▆▇████████▁▆███▄▇▇▇███▂▇▇█████████
val_loss,▂▂▁▁▃▃▁▁▁▁▁▁▁▁▁▁▃█▁▂▁▂▁▁▁▁▁▁▃▁▁▁▁▁▁▁▁▁▁▁
epoch,21
train_acc,0.99021
train_f1,0.98973
train_loss,0.06302


In [2]:
# Test 데이터셋에 대한 변환 정의
tst_transform = A.Compose([
    A.Resize(height=img_size, width=img_size),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(),
])

# 테스트 데이터 예측 및 결과 저장
test_dataset = ImageDataset(test_df_test, data_path_test, transform=tst_transform)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=num_workers, pin_memory=True)

# 폴드별 예측 결과 저장
fold_preds = []

for fold in range(n_splits):
    model = CustomSAR(num_classes=17, dropout_prob=DROPOUT_PROB).to(device)
    model.load_state_dict(torch.load(best_model_paths[fold]))
    model.eval()

    fold_pred = []

    # 테스트 데이터에 대한 예측
    with torch.no_grad():
        for images, _ in test_loader:
            images = images.to(device)
            output = model(images)
            fold_pred.extend(output.argmax(dim=1).cpu().numpy())

    fold_preds.append(np.array(fold_pred))

# 폴드별 예측을 모드 계산하여 최종 예측 생성
final_class_preds = mode(np.array(fold_preds), axis=0).mode[0]

# 결과 저장
submission = pd.DataFrame({'ID': test_df_test['ID'], 'target': final_class_preds})
submission.to_csv('submit_v6_x50_class_SAR_attention.csv', index=False)

# 최종 검증 성능 출력
mean_val_loss = np.mean([m[0] for m in fold_val_metrics])
mean_val_f1 = np.mean([m[1] for m in fold_val_metrics])

print(f"Mean Validation Loss: {mean_val_loss:.4f}, Mean Validation F1 Score: {mean_val_f1:.4f}")
print("최종 제출 파일이 저장되었습니다.")

Mean Validation Loss: 0.0358, Mean Validation F1 Score: 0.9918
최종 제출 파일이 저장되었습니다.


In [3]:
# 최종 검증 성능 출력
mean_val_acc = np.mean([m[2] for m in fold_val_metrics])

print(f"Mean Validation Accuracy: {mean_val_acc:.4f}")

IndexError: tuple index out of range